In [1]:
# Base functionality
using Plots, Measures, Statistics

# ODE Solver
using DifferentialEquations

# Benchmarking
using BenchmarkTools

include("../MixedMealModel.jl")

FitModelSampledLHC (generic function with 2 methods)

In [2]:
sample_person = (
    fasting_glucose = 5.,
    fasting_insulin = 18.,
    fasting_triglyceride = 1.3,
    fasting_NEFA = 0.33,
    body_weight = 84.2,
    meal_glucose = 75000.,
    meal_triglyceride = 60000.
)

parameters = InitialParameters(sample_person, k1 = 0.0164, k5=0.0564, 
k6 = 2.7341, k11 = 0.00035, k12 = 0.0822, tau_LPL = 187.88, 
k14 = 0.0392, k16 = 0.0135)

constants = Constants(parameters, sample_person);

In [3]:
tspan::Tuple{Float64, Float64} = (0., 500.)
u0 = InitialValues(sample_person)
MealModel! = MixedMealModel(constants, sample_person)
System = ODEProblem(MealModel!, u0, tspan, parameters)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 720.0)
u0: 13-element Vector{Float64}:
  0.0
  5.0
  0.0
 18.0
  0.0
 18.0
 18.0
 18.0
  0.33
  0.0
  0.0
  0.0
  1.3

In [4]:
fracs = range(0.5, 1.5, 50)

prob_func = let p = parameters
    (prob, i, repeat) -> begin
        remake(prob, p = [fracs[i]*p[1]; p[2:end]])
    end
end

#32 (generic function with 1 method)

In [5]:
ensemble_prob = EnsembleProblem(System, prob_func = prob_func)

EnsembleProblem with problem ODEProblem

In [8]:
BenchmarkResult = @benchmark solve(ensemble_prob, Tsit5(), EnsembleThreads(), trajectories = 50) samples=100 seconds=10

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  1.312 ms …  1.779 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.485 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.494 ms ± 96.475 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▂▄▆    ▂  ▄   █    ▂                          
  ▄▄▁▆▆▄▁██▄▆▄▆▆███▆█▄██▆██▆▁▄██▆▄▄█▁█▆▆▆▄█▁▁▁▄▁▄▄▄▄▁▁▁▁▁▁▁▄ ▄
  1.31 ms        Histogram: frequency by time        1.76 ms <

 Memory estimate: 7.41 MiB, allocs estimate: 49078.